Pipeline 中 if / switch 的「本质」是什么？
一句话核心：
👉 根据 Context 中的数据，决定下一个要执行的 Command / 子 Pipeline

In [2]:
from dataclasses import dataclass
from typing import Callable


@dataclass
class Context:
    data: dict


class Command:
    def run(self, ctx: Context) -> Context:
        return ctx


class IFCommand(Command):
    def __init__(
        self,
        condition: Callable[[Context], bool],
        true_cmd: Command,
        false_cmd: Command | None = None,
    ):
        self.condition = condition
        self.true_cmd = true_cmd
        self.false_cmd = false_cmd

    def run(self, ctx: Context) -> Context:
        if self.condition(ctx):
            return self.true_cmd.run(ctx)
        elif self.false_cmd:
            return self.false_cmd.run(ctx)
        return ctx


# 示例
# 是否登录
class Login(Command):
    def run(self, ctx):
        print("🔐 执行登录")
        ctx.data["logged_in"] = True
        return ctx


class SkipLogin(Command):
    def run(self, ctx):
        print("➡️ 已登录，跳过")
        return ctx


ctx = Context({"logged_in": False})
pipeline = IFCommand(
    condition=lambda ctx: not ctx.data["logged_in"],
    true_cmd=Login(),
    false_cmd=SkipLogin(),
)

pipeline.run(ctx)


🔐 执行登录


Context(data={'logged_in': True})

In [ ]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import Any, Callable, Protocol, runtime_checkable


# 改造成鸭子类型
@dataclass
class Context:
    data: dict[str, Any] = field(default_factory=dict)


@runtime_checkable
class Command(Protocol):
    """命令契约：只要实现 run(ctx)->ctx，就算 Command"""

    def run(self, ctx: Context) -> Context: ...


class IFCommand:
    """条件分支命令：依赖的是 Command Protocol，而非继承体系"""

    def __init__(
        self,
        condition: Callable[[Context], bool],
        true_cmd: Command,
        false_cmd: Command | None = None,
    ):
        self.condition = condition
        self.true_cmd = true_cmd
        self.false_cmd = false_cmd

    def run(self, ctx: Context) -> Context:
        if self.condition(ctx):
            return self.true_cmd.run(ctx)
        if self.false_cmd is not None:
            return self.false_cmd.run(ctx)
        return ctx


# 示例：不需要继承 Command，只要有 run 方法即可 ✅
class Login:
    def run(self, ctx: Context) -> Context:
        print("🔐 执行登录")
        ctx.data["logged_in"] = True
        return ctx


class SkipLogin:
    def run(self, ctx: Context) -> Context:
        print("➡️ 已登录，跳过")
        return ctx


def main() -> None:
    ctx = Context({"logged_in": False})

    pipeline = IFCommand(
        # 可调用对象
        condition=lambda ctx: not ctx.data["logged_in"],
        true_cmd=Login(),
        false_cmd=SkipLogin(),
    )

    # （可选）运行时检查：这对象是不是满足 Protocol
    assert isinstance(Login(), Command)

    pipeline.run(ctx)


In [ ]:
# 谓词
from typing import Callable

Predicate = Callable[[Context], bool]
# 组合器实现（可复用、可单测）


def AND(*preds: Predicate) -> Predicate:
    return lambda ctx: all(p(ctx) for p in preds)


def OR(*preds: Predicate) -> Predicate:
    return lambda ctx: any(p(ctx) for p in preds)


def NOT(pred: Predicate) -> Predicate:
    return lambda ctx: not pred(ctx)


# 业务
is_logged_in: Predicate = lambda ctx: bool(ctx.data.get("logged_in", False))
need_login: Predicate = lambda ctx: bool(ctx.data.get("need_login", True))
is_active: Predicate = lambda ctx: ctx.data.get("account_status") == "active"
session_expired: Predicate = lambda ctx: bool(ctx.data.get("session_expired", False))
force_login: Predicate = lambda ctx: bool(ctx.data.get("force_login", False))

# “必须都满足才登录”（all / AND）
cond = AND(NOT(is_logged_in), need_login, is_active)

# “有一个触发就登录”（any / OR）

cond = OR(force_login, session_expired, NOT(is_logged_in))

# 使用
pipeline = IFCommand(
    # 条件
    condition=cond,
    true_cmd=Login(),
    false_cmd=SkipLogin(),
)


In [ ]:
# 带短路 + 带解释（调试很爽）
from dataclasses import dataclass
from typing import Optional


@dataclass(frozen=True)
class Pred:
    name: str
    fn: Predicate

    def __call__(self, ctx: Context) -> bool:
        return self.fn(ctx)


def ALL(*preds: Pred) -> Predicate:
    def _cond(ctx: Context) -> bool:
        for p in preds:
            ok = p(ctx)
            if not ok:
                ctx.data["last_failed_pred"] = p.name  # 记录失败原因
                return False
        return True

    return _cond


def ANY(*preds: Pred) -> Predicate:
    def _cond(ctx: Context) -> bool:
        for p in preds:
            ok = p(ctx)
            if ok:
                return True
        return False

    return _cond


# 使用
cond = ALL(
    Pred("not_logged_in", lambda ctx: not ctx.data.get("logged_in", False)),
    Pred("need_login", lambda ctx: ctx.data.get("need_login", True)),
    Pred("account_active", lambda ctx: ctx.data.get("account_status") == "active"),
)


## 多分支


In [3]:
class SwitchCommand(Command):
    def __init__(
        self,
        selector: Callable[[Context], str],
        routes: dict[str, Command],
        default: Command | None = None,
    ):
        self.selector = selector
        self.routes = routes
        self.default = default

    def run(self, ctx: Context) -> Context:
        key = self.selector(ctx)
        cmd = self.routes.get(key, self.default)
        if cmd:
            return cmd.run(ctx)
        return ctx


class BilibiliTask(Command):
    def run(self, ctx):
        print("📺 执行 B站 任务")
        return ctx


class DouyinTask(Command):
    def run(self, ctx):
        print("🎵 执行 抖音 任务")
        return ctx


ctx = Context({"platform": "bilibili"})

pipeline = SwitchCommand(
    selector=lambda ctx: ctx.data["platform"],
    routes={
        "bilibili": BilibiliTask(),
        "douyin": DouyinTask(),
    },
)

pipeline.run(ctx)


📺 执行 B站 任务


Context(data={'platform': 'bilibili'})

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Protocol, runtime_checkable


# 1) Context：流水线的上下文
@dataclass
class Context:
    data: dict


# 2) Command：统一接口（用 Protocol 做“结构化类型”）
@runtime_checkable
class Command(Protocol):
    def run(self, ctx: Context) -> Context: ...


# 3) Predicate：条件函数的接口（也用 Protocol）
@runtime_checkable
class Predicate(Protocol):
    def __call__(self, ctx: Context) -> bool: ...


# @dataclass(frozen=True)
# class Pred:
#     name: str
#     fn: Predicate

#     def __call__(self, ctx: Context) -> bool:
#         return self.fn(ctx)


# 4) IFCommand：它自己也实现 Command => 可嵌套 ✅
@dataclass(frozen=True)
class IFCommand:
    condition: Predicate
    true_cmd: Command
    false_cmd: Command | None = None

    def run(self, ctx: Context) -> Context:
        if self.condition(ctx):
            return self.true_cmd.run(ctx)
        if self.false_cmd is not None:
            return self.false_cmd.run(ctx)
        return ctx


# ======== 示例命令 ========
class Login:
    def run(self, ctx: Context) -> Context:
        print("🔐 执行登录")
        ctx.data["logged_in"] = True
        return ctx


class SkipLogin:
    def run(self, ctx: Context) -> Context:
        print("➡️ 已登录，跳过登录")
        return ctx


class BindCookie:
    def run(self, ctx: Context) -> Context:
        print("🍪 绑定 cookie")
        ctx.data["cookie"] = "abc123"
        return ctx


class SkipBindCookie:
    def run(self, ctx: Context) -> Context:
        print("➡️ 已有 cookie，跳过绑定")
        return ctx


# ======== 嵌套 if 示例 ========
# cookie_missing = Pred("cookie_missing", lambda ctx: ctx.data.get("cookie") is None)

ctx = Context({"logged_in": False, "cookie": None})

pipeline: Command = IFCommand(
    condition=lambda ctx: ctx.data.get("logged_in", False),  # ✅ 创建条件函数
    true_cmd=IFCommand(  # ✅ true 分支里再嵌套 IFCommand
        condition=lambda ctx: ctx.data.get("cookie") is None,
        true_cmd=BindCookie(),
        false_cmd=SkipBindCookie(),
    ),
    false_cmd=SkipLogin(),
)

pipeline.run(ctx)
print("最终 ctx =", ctx)
